In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid
from clients.backtester import poloniex as px
from clients.backtester import helpers
from clients.backtester import gemini
from clients.pypoloniex.pypoloniex import LoadPairs, TimeSeries

## Data

In [ ]:
# Well This is a big joke, the library at the bottom makes all these useless

def get_ohlcv_fpath(market, coin, exchange, period):
    fname = '{:s}_{:s}_{:s}_{:d}.csv'.format(
        exchange, market, coin, period)
    return os.path.join(cfg.DATA_DIR, fname)

def download_absolute_prices(market, coin, exchange, period, start, end):
    """
    market = base currency
    start = '4/2/2014'       # dd/mm/year
    end =  '11/2/2014'       # dd/mm/year
    period = 86400           # candle stick period in seconds
    """
    sess = TimeSeries()
    pair = (market, coin)
    sess.getData(pair, period, start, end)
    sess.toCSV(get_ohlcv_fpath(market, coin, exchange, period))
    return sess

def download_relative_prices(market, coin, exchange, period, days_back=30, days_data=30):
    """
    pair = "USDT_BTC"    # Use ETH pricing data on the BTC market
    period = 1800       # Use 1800 second candles
    daysBack = 30       # Grab data starting 30 days ago
    daysData = 30       # From there collect 60 days of data
    """
    pair = '{:s}_{:s}'.format(market, coin)
    data = px.getPast(pair, period, days_back, days_data)
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.index = df['date']
    df.drop('date', axis=1, inplace=True)

    fpath = get_ohlcv_fpath(market, coin, exchange, period)
    df.to_csv(fpath, index=True)
    sess = TimeSeries()
    sess.fromCSV(fpath)
    return sess

def load_historical_prices(market, coin, exchange, period):
    sess = TimeSeries()
    fpath = get_ohlcv_fpath(market, coin, exchange, period)
    sess.fromCSV(fpath)
    return sess

## Cleaner

def download_prices(market, coin, exchange, period):
    #Params: String symbol, int frequency = 300,900,1800,7200,14400,86400
    #Returns: df from first available date
    ticker = '{:s}_{:s}'.format(market, coin)
    url = ('https://poloniex.com/public?command=returnChartData&currencyPair=' 
           + ticker + '&end=9999999999&period=' + str(period) + '&start=0')
    df = pd.read_json(url)
    df.set_index('date', inplace=True)
    fpath = get_ohlcv_fpath(market, coin, exchange, period)
    df.to_csv(fpath, index=True)
    print('Processed: ' + ticker, "Rows: ", len(df))
    return df    

def load_prices(market, coin, exchange, period):
    fpath = get_ohlcv_fpath(market, coin, exchange, period)
    df = pd.read_csv(fpath, index_col = 'date')
    df.dropna(inplace=True)
    df['date'] = [datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S') for t in df.index.values]
    df = df.set_index('date')
    return df

def download_bulk_prices(market, coins, exchange, period):
    for coin in coins:
        print("Downloading", coin)
        df = download_prices(market, coin, exchange, period)

def load_bulk_prices(market, coins, exchange, period):
    df = pd.DataFrame()
    for coin in coins:
        ticker = '{:s}_{:s}'.format(market, coin)
        df[ticker] = load_prices(market, coin, exchange, period)['close']
    df.dropna(inplace=True)
    return df

In [ ]:
# Config
coin = c.ETH
market = c.BTC
exchange = c.POLONIEX
days_back = 30
days_data = 30
start_time = '1/1/2017'
end_time = '1/1/2018'
period = 1800

In [ ]:
# Relative
relative = download_relative_prices(market, coin, exchange, period, 30, 30)
relative = load_historical_prices(market, coin, exchange, period)
relative.data['close'].plot()

In [ ]:
# Absolute
absolute = download_absolute_prices(market, coin, exchange, period, start_time, end_time)
absolute = load_historical_prices(market, coin, exchange, period)
absolute.data['close'].plot()

In [ ]:
# Download Bulk All Available Data
coins = [c.LTC, c.ETH, c.XRP, c.XMR, c.DASH]
tickers =  ['USDT_ETC','USDT_XMR','USDT_ETH','USDT_DASH',
 'USDT_XRP','USDT_LTC','USDT_NXT','USDT_STR','USDT_REP','USDT_ZEC']
coins = [t.split('_')[-1] for t in tickers]
market = c.BTC
download_bulk_prices(market, coins, exchange, period)

In [ ]:
# Load Bulk Prices
bulk_prices = load_bulk_prices(market, coins, exchange, period)
bulk_prices.head()

In [ ]:
# Check for missing timesteps
def check_missing_timesteps(df, timestep):
    start_time = df.index[0]
    end_time = df.index[-1]
    print("Start", start_time)
    print("End", end_time)
    last_time = start_time
    n_missing = 0
    for idx,row in df[1:].iterrows():
        cur_time = idx
        if cur_time != last_time + datetime.timedelta(seconds=timestep):
            print("Expected:", last_time + datetime.timedelta(seconds=timestep), 
                  "| Time:", cur_time)
            n_missing += (cur_time - last_time).seconds//timestep
        last_time = cur_time
    return n_missing

In [ ]:
#check_missing_timesteps(relative.data, period)
#check_missing_timesteps(absolute.data, period)
check_missing_timesteps(bulk_prices, period)

## Explore

In [ ]:
crypto_df = load_bulk_prices(market, coins, exchange, period)

In [ ]:
crypto_df_norm = crypto_df.divide(crypto_df.iloc[0])
crypto_df_pct = crypto_df.pct_change().dropna()
corr = crypto_df_pct.corr()
crypto_df_norm.plot(figsize=(18,8))

In [ ]:
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
plt.scatter(crypto_df_pct[market+'_DASH'],crypto_df_pct[market+'_XMR'])

In [ ]:
import statsmodels.api as sm
model = sm.OLS(crypto_df_pct[market+'_XMR'],
               crypto_df_pct[market+'_DASH']).fit()
model.summary()

In [ ]:
line=[model.params[0]*i for i in crypto_df_pct['USDT_DASH'].values]
plt.plot(crypto_df_pct['USDT_DASH'], line, c = 'r')
plt.scatter(crypto_df_pct['USDT_DASH'],crypto_df_pct['USDT_XMR'])
plt.xlabel('USDT_DASH % Return')
plt.ylabel('USDT_XMR % Return')

In [ ]:
coin = c.LTC
market = c.BTC
exchange = c.POLONIEX
period = 300
df = download_prices(market, coin, exchange, period)

In [ ]:
df[market+'_'+coin] = load_prices(market, coin, exchange, period)['close']
df['SMA_1000'] = df['close'].rolling(1000).mean()
df['SMA_5000'] = df['close'].rolling(5000).mean()
df[['close','SMA_1000','SMA_5000']][270000:].plot(figsize = (16,10))

In [ ]:
def test_ma(df, lead, lag, pc_thresh = 0.025):
    ma_df = df.copy()
    ma_df['lead'] = ma_df['close'].rolling(lead).mean()
    ma_df['lag'] = ma_df['close'].rolling(lag).mean()
    ma_df.dropna(inplace = True)
    ma_df['lead-lag'] = ma_df['lead'] - ma_df['lag']
    ma_df['pc_diff'] = ma_df['lead-lag'] / ma_df['close']
    ma_df['regime'] = np.where(ma_df['pc_diff'] > pc_thresh, 1, 0)
    ma_df['regime'] = np.where(ma_df['pc_diff'] < -pc_thresh, -1, ma_df['regime'])
    ma_df['Market'] = np.log(ma_df['close'] / ma_df['close'].shift(1))
    ma_df['Strategy'] = ma_df['regime'].shift(1) * ma_df['Market']
    ma_df[['Market','Strategy']] = ma_df[['Market','Strategy']].cumsum().apply(np.exp)
    return ma_df

In [ ]:
ma_df = test_ma(df, 1000, 5000).dropna()
ma_df['regime'].plot(figsize=(16,5))

In [ ]:
ma_df[['Market','Strategy']].iloc[-1]

In [ ]:
ma_df[['Market','Strategy']][200000:].plot(figsize = (16,10))

In [ ]:
leads = np.arange(100, 4100, 100)
lags = np.arange(4100, 8100, 100)
lead_lags = [[lead,lag] for lead in leads for lag in lags]
pnls = pd.DataFrame(index=lags,columns = leads)

In [ ]:
for lead, lag in lead_lags:
    pnls[lead][lag] = test_ma(df, lead, lag)['Strategy'][-1]
    #print(lead,lag,pnls[lead][lag])

In [ ]:
PNLs = pnls[pnls.columns].astype(float)
plt.subplots(figsize = (14,10))
sns.heatmap(PNLs, cmap='PiYG')
#PNLs.max()

In [ ]:
df.dropna().head()

## API

* https://poloniex.com/support/api/
* https://pypi.python.org/pypi/poloniex
* https://github.com/Aula13/poloniex
* https://github.com/s4w3d0ff/python-poloniex
* https://pastebin.com/fbkheaRb
* https://github.com/timucin/cyrpto_trader/blob/master/cyripto_trader.py
* https://github.com/s4w3d0ff/python-poloniex/blob/master/examples/chart/bokehPlotter.py (charts)

Primary Reference:

* https://github.com/s4w3d0ff/python-poloniex/blob/master/poloniex/__init__.py

In [ ]:
from poloniex import Poloniex
polo = Poloniex(cfg.POLONIEX_API_KEY, cfg.POLONIEX_API_SECRET_KEY)

In [83]:
# Public
currency_pair = 'BTC_ETH'
ticker = polo.returnTicker()[currency_pair]
print(ticker)

trade_history = polo.returnTradeHistoryPublic(currency_pair)
print(trade_history[:1])

print(polo.return24hVolume()[currency_pair])

{'id': 148, 'last': 0.05922985, 'lowestAsk': 0.05927996, 'highestBid': 0.05922985, 'percentChange': 0.09881004, 'baseVolume': 4793.86996323, 'quoteVolume': 85578.19441033, 'isFrozen': 0, 'high24hr': 0.05946706, 'low24hr': 0.05378583}
[{'globalTradeID': 312150098, 'tradeID': 39263987, 'date': '2018-01-02 01:38:39', 'type': 'sell', 'rate': 0.05922985, 'amount': 6.39170071, 'total': 0.37857947}]
{'BTC': 4793.86996323, 'ETH': 85578.19441033}


In [113]:
# Prices

def pair(market, coin):
    return market+'_'+coin

def fetch_chart_data(market, coin, period, start, end):
    data = polo.returnChartData(pair(market,coin), period, start, end)
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.index = df['date']
    df.drop('date', axis=1, inplace=True)
    return df    

def fetch_and_save_chart_data(market, coin, period, start, end):
    print("Downloading:", market, coin)
    df = fetch_chart_data(market, coin, period, start, end)
    fpath = get_ohlcv_fpath(market, coin, c.POLONIEX, period)
    df.to_csv(fpath, index=True)
    print("Downloaded rows:", len(df))
    return df

def load_chart_data(market, coin, exchange, period):
    fpath = get_ohlcv_fpath(market, coin, exchange, period)
    df = pd.read_csv(fpath, index_col = 'date')
    return df

def download_ohlcv(market, coins, period):
    for coin in coins:
        _ = fetch_and_save_chart_data(market, coin, period, start, end)

def load_currency_group_prices(market, coins, exchange, period):
    df = pd.DataFrame()
    for coin in coins:
        ticker = '{:s}_{:s}'.format(market, coin)
        df[ticker] = load_chart_data(market, coin, exchange, period)['close']
    df.dropna(inplace=True)
    return df

In [111]:
# Get historical data
coins = [c.BTC, c.LTC, c.ETH, c.XRP, c.XMR, c.DASH]
market = c.USDT
coin = c.ETH
period = 1800
start = datetime.datetime(year=2015, month=9, day=1).timestamp()
end = datetime.datetime(year=2018, month=1, day=1).timestamp()

In [109]:
#df = fetch_chart_data(market, coin, period, start, end)
df = fetch_and_save_chart_data(market, coin, period, start, end)
df = load_chart_data(market, coin, c.POLONIEX, period)
df.head()

Downloading:  BTC ETH
Downloaded rows: 40947


,close,high,low,open,quoteVolume,volume,weightedAverage
date,,,,,,,
2015-09-01 07:00:00,0.005956,0.005960,0.005910,0.005940,762.534086,4.518152,0.005925
2015-09-01 07:30:00,0.005901,0.005979,0.005901,0.005925,2965.341116,17.610972,0.005939
2015-09-01 08:00:00,0.005956,0.005957,0.005880,0.005940,1677.445981,9.922877,0.005915
2015-09-01 08:30:00,0.005900,0.005955,0.005897,0.005955,126.649142,0.750902,0.005929
2015-09-01 09:00:00,0.005939,0.005940,0.005898,0.005901,2118.748940,12.522945,0.005911


In [112]:
download_ohlcv(market, coins, period)

Downloading:  USDT BTC
Downloaded rows: 40947
Downloading:  USDT LTC
Downloaded rows: 40947
Downloading:  USDT ETH
Downloaded rows: 40947
Downloading:  USDT XRP
Downloaded rows: 40947
Downloading:  USDT XMR
Downloaded rows: 40947
Downloading:  USDT DASH
Downloaded rows: 40947


In [114]:
df = load_currency_group_prices(market, coins, c.POLONIEX, period)
df.head()

,USDT_BTC,USDT_LTC,USDT_ETH,USDT_XRP,USDT_XMR,USDT_DASH
date,,,,,,
2015-09-01 07:00:00,226.11,2.469939,1.34,0.005508,0.490133,2.699994
2015-09-01 07:30:00,226.11,2.469939,1.34,0.009536,0.490133,2.699994
2015-09-01 08:00:00,226.11,2.469939,1.34,0.009536,0.490133,2.699994
2015-09-01 08:30:00,226.11,2.469939,1.34,0.009536,0.490133,2.699994
2015-09-01 09:00:00,229.50,2.469939,1.34,0.009536,0.490133,2.699994


In [ ]:
# Private

print(polo.returnBalances())
print(polo.returnTradableBalances())
print(polo.returnTradeHistory())
print(polo.returnOpenOrders(currencyPair='all'))
print(polo.returnFeeInfo())
print(polo.returnAvailableAccountBalances())
print(polo.returnMarginAccountSummary())

In [ ]:
# Trading
# https://github.com/timucin/cyrpto_trader/blob/master/cyripto_trader.py
resp = polo.buy(pair, rate, amount, fillOrKill=None,
                immediateOrCancel=None, postOnly=None)
print(resp)

resp = polo.sell(pair, rate, amount, fillOrKill=None,
                 immediateOrCancel=None, postOnly=None)
print(resp)

In [ ]:
# Manage orders
resp = polo.cancelOrder(orderNumber)
print(resp)

resp = polo.moveOrder(orderNumber, rate, amount=None, 
                      postOnly=None, immediateOrCancel=None)
print(resp)
resp = polo.transferBalance(currency, amount, fromAccount, toAccount)
print(resp)

In [ ]:
# Margin Trading
resp = polo.marginBuy(currencyPair, rate, amount, lendingRate)
print(resp)

resp = polo.marginSell(currencyPair, rate, amount, lendingRate)
print(resp)

print(polo.getMarginPosition(currencyPair))
print(polo.closeMarginPosition(currencyPair))